In [12]:
from paddle.vision import models
from paddle import summary
import paddle
from paddle.nn import Linear
net = models.resnet18(pretrained=True)
net.fc = Linear(in_features=512,out_features=176)
def init_layer(layer):
    if isinstance(layer,(Linear)):
        paddle.nn.init.kaiming_normal_(layer.weight)
        paddle.nn.init.constant_(layer.bias,0)
init_layer(net.fc)

/home/ricaedo/下载/yes/envs/paddle/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
W1117 16:01:28.189061  8552 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.9, Driver API Version: 12.8, Runtime API Version: 12.6


In [13]:
! pip install pandas
import pandas as pd 
import numpy as np


In [14]:
train_mapping = pd.read_csv("./train.csv")
test_mapping = pd.read_csv("./test.csv")
result = train_mapping["label"].value_counts()
index_list  = result.index.to_list()
#value_list  = result.values.tolist()
mapping_toindex = {}
mapping_tolabel = {}
for index , label in enumerate(index_list):
    mapping_tolabel[index] = label
for index,label in enumerate(index_list):
    mapping_toindex[label] = index

import gc
del result
del index_list
gc.collect
len(mapping_toindex)

176

In [15]:
from PIL import Image
from paddle.io import Dataset
from paddle.vision import transforms
class mydataset(Dataset):
    def __init__(self,originDataset,mapping,transformImage):
        super().__init__()
        self.originDataset = originDataset
        self.mapping = mapping
        self.transformImage = transformImage
        
    def __getitem__(self, idx):
        image_path = self.originDataset.iloc[idx,0]
        image_PIL = Image.open(image_path).convert("RGB")  
        try:
            originalLabel = self.originDataset.iloc[idx,1] 
            return self.transformImage(image_PIL),self.mapping[originalLabel]
        except Exception:
            return self.transformImage(image_PIL)
    def __len__(self):

        return len(self.originDataset)
    


In [16]:
paddle.disable_static()
from paddle.io import random_split
transformImageTest = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],
                             std=[0.229,0.224,0.225])
    ]
)
transformImageTrain = transforms.Compose(
    [   
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        #transforms.ColorJitter(brightness=0.2,contrast=0.1,saturation=0.1,hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],
                             std=[0.229,0.224,0.225])
    ]
)
originalDataset = mydataset(train_mapping,mapping=mapping_toindex,transformImage=transformImageTrain)
length = len(originalDataset)
ds1 = int(0.9*length)
ds2 = length - ds1
trainDataset , valDataset = random_split(originalDataset,[ds1,ds2])
del originalDataset,ds1,ds2
gc.collect
testDataset = mydataset(test_mapping,mapping=mapping_toindex,transformImage=transformImageTest)

In [17]:
def val(valDataloader,model):
    model.eval()
    with paddle.no_grad():
        flag = 0
        acc = 0
        for features , labels in valDataloader:
            flag+=1
            y_hat = model(features)
            index_list = paddle.argmax(y_hat,axis=1)
            acc += (index_list == labels).float().mean()
        return acc/flag

In [18]:

device = paddle.get_device()

In [19]:
config = {
    "lossfunction" : "CrossEntropyLoss",
    "optim" : "Adam",
    "n_epoch" : 20,
    "lr" : 0.0001,
    "weight_deacy" : 1e-4,
    "batchSize" : 32,
    "val_epoch" : 10,
    "save_predict_path" : "./submission.csv"
}

In [20]:
from paddle.io import DataLoader
def get_loader(config,dataset,mode):

    return DataLoader(
        dataset=dataset,
        batch_size=config["batchSize"],
        shuffle=(mode == "train"),
        drop_last=(mode == "train") 
    )
trainDataloader = get_loader(dataset=trainDataset,config=config,mode="train")
valDataloder = get_loader(dataset=valDataset,config=config,mode="val")
testDataloder = get_loader(dataset=testDataset,config=config,mode="test")


In [21]:
! pip install tqdm
from tqdm.auto import tqdm
def train(config,trainDataloader,valDataloder,device,model):
    paddle.device.set_device(device=device)
    model.train()
    n_epoch = config["n_epoch"]
    params_lx = [
        param for name , param in model.named_parameters() 
            if name not in ["fc.weight","fc.bias"]
    ]
    if params_lx:
        optimer = getattr(paddle.optimizer,config["optim"])(
            parameters = [
                {
                "params":params_lx,
                "lr_mult":1.0,
                }
                ,
                {
                "params":list(model.fc.parameters()),
                "lr_mult" : 10.0,
                }
            ],
            learning_rate = config["lr"],
            weight_decay = config["weight_deacy"]
            )
    else:
        optimer = getattr(paddle.optimizer,config["optim"])(
            model.parameters(),
            learning_rate = config["lr"],
            weight_decay = config["weight_deacy"],
        )
    lossfunction = getattr(paddle.nn,config["lossfunction"])()
    val_epoch = config["val_epoch"]
    limit_acc = 0
    stoppingEpoch = 0
    
    for _ in tqdm(range(n_epoch)):
        accy = 0
        f = 0
        for features , labels in tqdm(trainDataloader):
            f+=1
            y_hat = model(features)
            loss = lossfunction(y_hat,labels)
            loss.backward()
            optimer.step()
            optimer.clear_grad()
            index_list = paddle.argmax(y_hat,axis=1)
            accy += (index_list == labels).float().mean()
        print("train",accy/f *100)
        acc = val(valDataloader=valDataloder,model=model)
        print("val:",acc*100)
        if acc>limit_acc:
            stoppingEpoch = 0
            limit_acc = acc
        else:
            stoppingEpoch +=1
        if stoppingEpoch >= val_epoch:
            break
        
train(config=config,trainDataloader=trainDataloader,valDataloder=valDataloder,device=device,model=net)

        

/home/ricaedo/下载/yes/envs/paddle/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  0%|          | 0/20 [00:00<?, ?it/s]/home/ricaedo/下载/yes/envs/paddle/lib/python3.10/site-packages/paddle/nn/layer/norm.py:818: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(
100%|██████████| 516/516 [00:47<00:00, 10.96it/s]


train Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       53.14316559)


  5%|▌         | 1/20 [00:49<15:34, 49.20s/it]

val: Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       77.08332825)


100%|██████████| 516/516 [00:51<00:00, 10.00it/s]


...

val: Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       92.49281311)


100%|██████████| 516/516 [00:51<00:00, 10.08it/s]


train Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       96.95373535)


100%|██████████| 20/20 [18:00<00:00, 54.04s/it]

val: Tensor(shape=[], dtype=float32, place=Place(gpu:0), stop_gradient=True,
       91.32543182)


In [25]:
def test(testDataloader,model):
    model.eval()
    with paddle.no_grad():
        predict = []
        for test_features in testDataloader:
            y_hat = model(test_features)
            index_list = paddle.argmax(y_hat,axis=1)
            for i in index_list:
                predict.append(mapping_tolabel[int(i)])
    return predict
def save_reslut(config,predict):
    save_predict_path = config["save_predict_path"]
    s = pd.read_csv("test.csv")
    y = s['image'].values.tolist()
    df = pd.DataFrame({
        'image': y,
        'label': predict  # 确保preds是一维数组
    })
    df.to_csv(save_predict_path, index=False)
result = test(testDataloader=testDataloder,model=net)
save_reslut(config=config,
            predict=result)
